#### 显示手写体数字图片经PCA压缩后的二维空间分布

In [5]:
# 导入pandas用于数据读取和处理
import pandas as pd

import numpy as np

#从互联网读入手写体图片识别任务的训练数据，存储在变量digits_train中。
digits_train=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tra', header=None)

#从互联网读入手写体图片识别任务的测试数据，存储在变量digits_test中。
digits_test=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tes', header=None)

#分割训练数据的特种向量和标记。
X_digits=digits_train[np.arange(64)]
y_digits=digits_train[64]

# 从sklearn.decomposition导入PCA。
from sklearn.decomposition import PCA

# 初始化一个可以将高维度特征向量（六十四维）压缩至二个维度的PCA。
estimator=PCA(n_components=2)
X_pca=estimator.fit_transform(X_digits)

#显示10类手写体数字图片经PCA压缩后的2维空间分布。
from matplotlib import pyplot as plt

def plot_pca_scatter():
    colors=['black','blue','purple','yellow','white','red','lime','cyan','orange','gray']
    for i in range(len(colors)):
        px=X_pca[:,0][y_digits.as_matrix()==i]
        py=X_pca[:,1][y_digits.as_matrix()==i]
        plt.scatter(px,py,c=colors[i])
        
    plt.legend(np.arange(0,10).astype(str))
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    plt.show()
    
plot_pca_scatter()


#### 使用原始像素特征和经PCA压缩重建的低维特征，在相同配置的支持向量机（分类）模型上分别进行图像识别

In [6]:
# 对训练数据、测试数据进行特征向量（图片像素）与分类目标的分隔。
X_train=digits_train[np.arange(64)]
y_train=digits_train[64]
X_test=digits_test[np.arange(64)]
y_test=digits_test[64]

# 导入基于线性核的支持向量机分类器。
from sklearn.svm import LinearSVC

# 使用默认配置初始化LinearSVC，对原始六十四维像素特征的训练数据进行建模，并在测试书上做预测，存储在y_predict中。
svc=LinearSVC()
svc.fit(X_train, y_train)
y_predict=svc.predict(X_test)

# 使用PCA将原六十四维的图像数据压缩到20个维度。
estimator=PCA(n_components=20)

#利用训练特征决定（fit）20个正交维度的方向，并转化（transform）原训练特征。
pca_X_train=estimator.fit_transform(X_train)
#测试特征也按照上述的20个正交维度方向进行转化（transform）。
pca_X_test=estimator.transform(X_test)

# 使用默认配置初始化LinearSVC,对压缩过后的二十维特征的训练数据进行建模，并在测试数据上做出预测，存储在pca_y_predict中。
pca_svc=LinearSVC()
pca_svc.fit(pca_X_train,y_train)
pca_y_predict=pca_svc.predict(pca_X_test)

#### 原始像素特征与PCA压缩重建的低维特征，在相同配置的支持向量机（分类）模型上识别性能的差异。

In [7]:
# 从sklearn.metrics导入classification_report用于更加细致的分类性能分析。
from sklearn.metrics import classification_report

# 对使用原始图像高维像素特征训练的支持向量机分类器的性能作出评估。
print svc.score(X_test, y_test)
print classification_report(y_test,y_predict,target_names=np.arange(10).astype(str))

# 对使用PCA压缩重建的低维图像特征训练的支持向量机分类器的性能作出评估
print pca_svc.score(pca_X_test, y_test)
print classification_report(y_test, pca_y_predict, target_names=np.arange(10).astype(str))


0.9271007234279355
             precision    recall  f1-score   support

          0       1.00      0.98      0.99       178
          1       0.80      0.97      0.88       182
          2       0.99      0.97      0.98       177
          3       0.99      0.90      0.94       183
          4       0.96      0.96      0.96       181
          5       0.91      0.96      0.94       182
          6       0.99      0.97      0.98       181
          7       0.99      0.88      0.93       179
          8       0.96      0.75      0.84       174
          9       0.77      0.93      0.84       180

avg / total       0.94      0.93      0.93      1797

0.9137451307735114
             precision    recall  f1-score   support

          0       0.97      0.96      0.96       178
          1       0.83      0.85      0.84       182
          2       0.95      0.97      0.96       177
          3       0.93      0.89      0.91       183
          4       0.93      0.98      0.95       181
    